In [1]:
# magics
%load_ext autoreload
%autoreload 2

# Imports

In [2]:
# Regular Imports
import os, sys, configparser
import time


In [3]:
# Scientific imports
from dask.distributed import Client, LocalCluster
from dask_jobqueue import SGECluster


/dls/science/groups/i04-1/conor_dev/ccp4/base/lib/python2.7/site-packages/dask_jobqueue/config.py:12: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [4]:
# Multi Dataset Crystalography imports
import multi_dataset_crystalography as mdc # import MultiCrystalDataset
from multi_dataset_crystalography.utils import DefaultPanDDADataloader


In [5]:
# PanDDA imports
from pandda_analyse.config import PanDDAConfig
from pandda_analyse.event_model import PanDDAEventModel
# from pandda_analyse.processor import ProcessModelSeriel
from pandda_analyse.event_model_distributed import PanDDAEventModelDistributed, load, fit, evaluate, criticise



# Args

In [6]:
# Arguments
arguments = None
# Config
config_path = "/dls/science/groups/i04-1/conor_dev/pandda/lib-python/pandda/pandda_analyse/pandda_analyse/analyse_config.ini"
config = configparser.ConfigParser()


# Config

In [7]:
config.read(config_path)

['/dls/science/groups/i04-1/conor_dev/pandda/lib-python/pandda/pandda_analyse/pandda_analyse/analyse_config.ini']

In [8]:
pandda_config = PanDDAConfig(config)  # Maps options to code abstractions

# Dataset

In [9]:
# Get Dataset
pandda_dataset = mdc.dataset.dataset.MultiCrystalDataset(dataloader=pandda_config.dataloader,
                                         sample_loader=pandda_config.sample_loader
                                        )

# Get reference Model

In [10]:
# Get reference model
reference = pandda_config.get_reference(pandda_dataset.datasets)
pandda_dataset.sample_loader.reference = reference

# Apply transforms to dataset

In [11]:
# Apply dataset transforms
if "data_check" in pandda_config.dataset_transforms:
    transform = pandda_config.dataset_transforms["data_check"]
    dataset = transform(pandda_dataset, reference)
    print("###### {} ######".format(transform.name))
    for block, record in transform.log().items():
        print("# # {} # #".format(block))
        print(record)

###### PanddaDataChecker ######
# # Rejected datasets # #
Datasets rejected: 
PDK2-x0279: rejected - rmsd to reference
PDK2-x0251: rejected - rmsd to reference
PDK2-x0107: rejected - rmsd to reference
PDK2-x0238: rejected - rmsd to reference
PDK2-x0878: rejected - rmsd to reference



In [12]:
if "scale_diffraction" in pandda_config.dataset_transforms:
    transform = pandda_config.dataset_transforms["scale_diffraction"]
    dataset = transform(dataset, reference)
    print("###### {} ######".format(transform.name))
    for block, record in transform.log().items():
        print("# # {} # #".format(block))
        print(record)

/dls/science/groups/i04-1/conor_dev/ccp4/base/lib/python2.7/site-packages/numpy/lib/function_base.py:3826: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


###### PanddaDiffractionScaler ######
# # Rejected datasets # #
Datasets rejected: 
PDK2-x0641: rejected - scaling failed
PDK2-x0318: rejected - scaling failed
PDK2-x0317: rejected - scaling failed



In [13]:
if "filter_structure" in pandda_config.dataset_transforms:
    transform = pandda_config.dataset_transforms["filter_structure"]
    dataset = transform(dataset, reference)
    print("###### {} ######".format(transform.name))
    for block, record in transform.log().items():
        print("# # {} # #".format(block))
        print(record)

###### PanddaDatasetFilterer ######
# # Rejected datasets # #
Datasets rejected: 
PDK2-x0281: rejected - non-identical structures
PDK2-x0365: rejected - non-identical structures
PDK2-x0658: rejected - non-identical structures
PDK2-x0288: rejected - non-identical structures
PDK2-x0182: rejected - non-identical structures
PDK2-x0347: rejected - non-identical structures
PDK2-x0676: rejected - non-identical structures
PDK2-x0585: rejected - non-identical structures
PDK2-x0248: rejected - non-identical structures
PDK2-x0106: rejected - non-identical structures
PDK2-x0287: rejected - non-identical structures
PDK2-x0229: rejected - non-identical structures
PDK2-x0289: rejected - non-identical structures
PDK2-x0299: rejected - non-identical structures
PDK2-x0310: rejected - non-identical structures
PDK2-x0316: rejected - non-identical structures
PDK2-x0192: rejected - non-identical structures
PDK2-x0050: rejected - non-identical structures
PDK2-x0236: rejected - non-identical structures
PDK2-x

In [14]:
if "filter_wilson" in pandda_config.dataset_transforms:
    transform = pandda_config.dataset_transforms["filter_wilson"]
    dataset = transform(dataset, reference)
    print("###### {} ######".format(transform.name))
    for block, record in transform.log().items():
        print("# # {} # #".format(block))
        print(record)

/home/zoh22914/.local/lib/python2.7/site-packages/panddas-0.2.12-py2.7.egg/bamboo/stats/__init__.py:11: RuntimeWarning: invalid value encountered in double_scalars
  return 0.6745*devs/mdev


###### PanddaDatasetFiltererWilsonRMSD ######
# # Rejected datasets # #
Datasets rejected: 




multi_dataset_crystalography/dataset/transforms.py:338: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  col='scaled_wilson_rmsd_all_z') > self.max_wilson_plot_z_score) or \
multi_dataset_crystalography/dataset/transforms.py:340: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  col='scaled_wilson_rmsd_<4A_z') > self.max_wilson_plot_z_score) or \
multi_dataset_crystalography/dataset/transforms.py:342: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  col='scaled_wilson_rmsd_>4A_z') > self.max_wilson_plot_z_score) or \
multi_dataset_crystalography/dataset/transforms.py:344: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  col='scaled_wilson_ln_rmsd_z') > self.max_wilson_plot_z_score

In [15]:
if "align" in pandda_config.dataset_transforms:
    transform = pandda_config.dataset_transforms["align"]
    dataset = transform(dataset, reference)
    print("###### {} ######".format(transform.name))
    for block, record in transform.log().items():
        print("# # {} # #".format(block))
        print(record)

[Parallel(n_jobs=15)]: Using backend LokyBackend with 15 concurrent workers.
[Parallel(n_jobs=15)]: Done   1 tasks      | elapsed:    6.7s
[Parallel(n_jobs=15)]: Done   2 tasks      | elapsed:    9.3s
[Parallel(n_jobs=15)]: Done   3 tasks      | elapsed:   11.5s
[Parallel(n_jobs=15)]: Done   4 tasks      | elapsed:   14.2s
[Parallel(n_jobs=15)]: Done   5 tasks      | elapsed:   17.1s
[Parallel(n_jobs=15)]: Done   6 tasks      | elapsed:   19.7s
[Parallel(n_jobs=15)]: Done   7 tasks      | elapsed:   22.3s
[Parallel(n_jobs=15)]: Done   8 tasks      | elapsed:   24.9s
[Parallel(n_jobs=15)]: Done   9 tasks      | elapsed:   27.3s
[Parallel(n_jobs=15)]: Done  10 tasks      | elapsed:   30.0s
[Parallel(n_jobs=15)]: Done  11 tasks      | elapsed:   32.4s
[Parallel(n_jobs=15)]: Done  12 tasks      | elapsed:   34.9s
[Parallel(n_jobs=15)]: Done  13 tasks      | elapsed:   37.5s
[Parallel(n_jobs=15)]: Done  14 tasks      | elapsed:   40.2s
[Parallel(n_jobs=15)]: Done  15 tasks      | elapsed:  

[Parallel(n_jobs=15)]: Done 132 tasks      | elapsed:   52.5s
[Parallel(n_jobs=15)]: Done 133 tasks      | elapsed:   52.6s
[Parallel(n_jobs=15)]: Done 134 tasks      | elapsed:   52.7s
[Parallel(n_jobs=15)]: Done 135 tasks      | elapsed:   52.7s
[Parallel(n_jobs=15)]: Done 136 tasks      | elapsed:   52.8s
[Parallel(n_jobs=15)]: Done 137 tasks      | elapsed:   52.9s
[Parallel(n_jobs=15)]: Done 138 tasks      | elapsed:   52.9s
[Parallel(n_jobs=15)]: Done 139 tasks      | elapsed:   52.9s
[Parallel(n_jobs=15)]: Done 140 tasks      | elapsed:   53.0s
[Parallel(n_jobs=15)]: Done 141 tasks      | elapsed:   53.0s
[Parallel(n_jobs=15)]: Done 142 tasks      | elapsed:   53.1s
[Parallel(n_jobs=15)]: Done 143 tasks      | elapsed:   53.1s
[Parallel(n_jobs=15)]: Done 144 tasks      | elapsed:   53.3s
[Parallel(n_jobs=15)]: Done 145 tasks      | elapsed:   53.3s
[Parallel(n_jobs=15)]: Done 146 tasks      | elapsed:   53.4s
[Parallel(n_jobs=15)]: Done 147 tasks      | elapsed:   53.8s
[Paralle

[Parallel(n_jobs=15)]: Done 268 tasks      | elapsed:  1.1min
[Parallel(n_jobs=15)]: Done 269 tasks      | elapsed:  1.1min
[Parallel(n_jobs=15)]: Done 270 tasks      | elapsed:  1.1min
[Parallel(n_jobs=15)]: Done 271 tasks      | elapsed:  1.1min
[Parallel(n_jobs=15)]: Done 272 tasks      | elapsed:  1.1min
[Parallel(n_jobs=15)]: Done 273 tasks      | elapsed:  1.1min
[Parallel(n_jobs=15)]: Done 274 tasks      | elapsed:  1.1min
[Parallel(n_jobs=15)]: Done 275 tasks      | elapsed:  1.1min
[Parallel(n_jobs=15)]: Done 276 tasks      | elapsed:  1.1min
[Parallel(n_jobs=15)]: Done 277 tasks      | elapsed:  1.1min
[Parallel(n_jobs=15)]: Done 278 tasks      | elapsed:  1.1min
[Parallel(n_jobs=15)]: Done 279 tasks      | elapsed:  1.1min
[Parallel(n_jobs=15)]: Done 280 tasks      | elapsed:  1.1min
[Parallel(n_jobs=15)]: Done 281 tasks      | elapsed:  1.1min
[Parallel(n_jobs=15)]: Done 282 tasks      | elapsed:  1.1min
[Parallel(n_jobs=15)]: Done 283 tasks      | elapsed:  1.1min
[Paralle

[Parallel(n_jobs=15)]: Done 404 tasks      | elapsed:  1.3min
[Parallel(n_jobs=15)]: Done 405 tasks      | elapsed:  1.3min
[Parallel(n_jobs=15)]: Done 406 tasks      | elapsed:  1.3min
[Parallel(n_jobs=15)]: Done 407 tasks      | elapsed:  1.3min
[Parallel(n_jobs=15)]: Done 408 tasks      | elapsed:  1.3min
[Parallel(n_jobs=15)]: Done 409 tasks      | elapsed:  1.3min
[Parallel(n_jobs=15)]: Done 410 tasks      | elapsed:  1.3min
[Parallel(n_jobs=15)]: Done 411 tasks      | elapsed:  1.3min
[Parallel(n_jobs=15)]: Done 412 tasks      | elapsed:  1.3min
[Parallel(n_jobs=15)]: Done 413 tasks      | elapsed:  1.3min
[Parallel(n_jobs=15)]: Done 414 tasks      | elapsed:  1.3min
[Parallel(n_jobs=15)]: Done 415 tasks      | elapsed:  1.3min
[Parallel(n_jobs=15)]: Done 416 tasks      | elapsed:  1.3min
[Parallel(n_jobs=15)]: Done 417 tasks      | elapsed:  1.3min
[Parallel(n_jobs=15)]: Done 418 tasks      | elapsed:  1.3min
[Parallel(n_jobs=15)]: Done 419 tasks      | elapsed:  1.3min
[Paralle

[Parallel(n_jobs=15)]: Done 541 tasks      | elapsed:  1.5min
[Parallel(n_jobs=15)]: Done 542 tasks      | elapsed:  1.5min
[Parallel(n_jobs=15)]: Done 543 tasks      | elapsed:  1.5min
[Parallel(n_jobs=15)]: Done 544 tasks      | elapsed:  1.5min
[Parallel(n_jobs=15)]: Done 545 tasks      | elapsed:  1.5min
[Parallel(n_jobs=15)]: Done 546 tasks      | elapsed:  1.5min
[Parallel(n_jobs=15)]: Done 547 tasks      | elapsed:  1.5min
[Parallel(n_jobs=15)]: Done 548 tasks      | elapsed:  1.5min
[Parallel(n_jobs=15)]: Done 549 tasks      | elapsed:  1.5min
[Parallel(n_jobs=15)]: Done 550 tasks      | elapsed:  1.5min
[Parallel(n_jobs=15)]: Done 551 tasks      | elapsed:  1.5min
[Parallel(n_jobs=15)]: Done 552 tasks      | elapsed:  1.5min
[Parallel(n_jobs=15)]: Done 553 tasks      | elapsed:  1.5min
[Parallel(n_jobs=15)]: Done 554 tasks      | elapsed:  1.5min
[Parallel(n_jobs=15)]: Done 555 tasks      | elapsed:  1.5min
[Parallel(n_jobs=15)]: Done 556 tasks      | elapsed:  1.5min
[Paralle

[Parallel(n_jobs=15)]: Done 675 tasks      | elapsed:  1.7min
[Parallel(n_jobs=15)]: Done 676 tasks      | elapsed:  1.7min
[Parallel(n_jobs=15)]: Done 677 tasks      | elapsed:  1.7min
[Parallel(n_jobs=15)]: Done 678 tasks      | elapsed:  1.7min
[Parallel(n_jobs=15)]: Done 679 tasks      | elapsed:  1.7min
[Parallel(n_jobs=15)]: Done 680 tasks      | elapsed:  1.7min
[Parallel(n_jobs=15)]: Done 681 tasks      | elapsed:  1.7min
[Parallel(n_jobs=15)]: Done 682 tasks      | elapsed:  1.7min
[Parallel(n_jobs=15)]: Done 683 tasks      | elapsed:  1.7min
[Parallel(n_jobs=15)]: Done 684 tasks      | elapsed:  1.7min
[Parallel(n_jobs=15)]: Done 685 tasks      | elapsed:  1.7min
[Parallel(n_jobs=15)]: Done 686 tasks      | elapsed:  1.7min
[Parallel(n_jobs=15)]: Done 687 tasks      | elapsed:  1.7min
[Parallel(n_jobs=15)]: Done 688 tasks      | elapsed:  1.7min
[Parallel(n_jobs=15)]: Done 689 tasks      | elapsed:  1.7min
[Parallel(n_jobs=15)]: Done 690 tasks      | elapsed:  1.7min
[Paralle

###### PanddaDefaultStructureAligner ######
# # aligned_datasets # #
Datasets aligned: 
PDK2-x0811
PDK2-x0621
PDK2-x0810
PDK2-x0082
PDK2-x0083
PDK2-x0080
PDK2-x0081
PDK2-x0086
PDK2-x0087
PDK2-x0084
PDK2-x0085
PDK2-x0815
PDK2-x0088
PDK2-x0089
PDK2-x0814
PDK2-x0165
PDK2-x0164
PDK2-x0167
PDK2-x0166
PDK2-x0161
PDK2-x0160
PDK2-x0163
PDK2-x0162
PDK2-x0245
PDK2-x0247
PDK2-x0240
PDK2-x0241
PDK2-x0242
PDK2-x0243
PDK2-x0819
PDK2-x0627
PDK2-x0788
PDK2-x0750
PDK2-x0400
PDK2-x0051
PDK2-x0401
PDK2-x0047
PDK2-x0019
PDK2-x0018
PDK2-x0786
PDK2-x0011
PDK2-x0010
PDK2-x0013
PDK2-x0012
PDK2-x0015
PDK2-x0014
PDK2-x0017
PDK2-x0016
PDK2-x0420
PDK2-x0421
PDK2-x0422
PDK2-x0424
PDK2-x0425
PDK2-x0427
PDK2-x0429
PDK2-x0540
PDK2-x0058
PDK2-x0538
PDK2-x0539
PDK2-x0684
PDK2-x0685
PDK2-x0686
PDK2-x0687
PDK2-x0680
PDK2-x0681
PDK2-x0729
PDK2-x0683
PDK2-x0727
PDK2-x0726
PDK2-x0725
PDK2-x0724
PDK2-x0688
PDK2-x0689
PDK2-x0721
PDK2-x0720
PDK2-x0334
PDK2-x0335
PDK2-x0336
PDK2-x0337
PDK2-x0330
PDK2-x0331
PDK2-x0332
PDK2-x0531

# Setup Output

In [16]:
tree = pandda_config.pandda_output(dataset)
for block, record in pandda_config.pandda_output.log().items():
    print("####### {} ########".format(block))
    print(record)

####### pandda_created ########
Output directory created: 
True



# Model

In [17]:
# Define Model
pandda_event_model = PanDDAEventModel(pandda_config.statistical_model,
                                      pandda_config.clusterer,
                                      pandda_config.event_finder,
                                      bdc_calculator=pandda_config.bdc_calculator,
                                      statistics=[],
                                      map_maker=pandda_config.map_maker, 
                                      event_table_maker=pandda_config.event_table_maker,
                                      cpus=config["args"]["cpus"],
                                      tree=tree)

# Partition

In [18]:
dataset.partitions = pandda_config.partitioner(dataset.datasets)

# Fit, evaluate, Criticise - Single dataset, Direct eval

##  VALIDATED!

In [19]:
# # Main Loop
# dataloader = DefaultPanDDADataloader(min_train_datasets=60, 
#                                      max_test_datasets=60)


In [20]:
# idx, ds = dataloader(dataset).next()

In [21]:
# context = pandda_event_model(idx, ds, reference)

In [22]:
# model = context.__enter__()

In [23]:
# model.fit()  # Fit the statistical model


In [24]:
# dtags = model.dataset.partition_samples("test").keys()
# truncated_datasets = model.dataset.sample_loader.truncated_datasets
# sample_loaders = {dtag: lambda d: model.dataset.sample_loader.get_sample(2.13786674777, d)
#                   for dtag
#                   in dtags}

In [25]:
# result = model.evaluate_single(sample_loaders["PDK2-x0621"],
#                                truncated_datasets["PDK2-x0621"],
#                                model.dataset.sample_loader.ref_map)

In [26]:
# model.grid = model.dataset.sample_loader.grid
# model.grid

In [27]:
# model.criticise_single(sample_loaders["PDK2-x0621"],
#                        truncated_datasets["PDK2-x0621"],
#                        model.dataset.sample_loader.ref_map,
#                        result[2],
#                        result[3],
#                        tree)

In [28]:
# model.dataset.max_res = max([d.data.summary.high_res for dtag, d in model.dataset.datasets.items()])

In [29]:
# model.evaluate_all()

In [30]:
# model.criticise_all(tree)

# Fit, evaluate, Criticise - All datasets, Local dask cluster

## TESTING

In [31]:
# # Main Loop
# dataloader = DefaultPanDDADataloader(min_train_datasets=60, 
#                                      max_test_datasets=60)


In [32]:
# ds = [(idx, d) for idx, d in dataloader(dataset)]

In [33]:
# # Set up client
# # client = dask.distributed.Client(scheduler_file="scheduler.json")
# cluster = LocalCluster(n_workers=2, threads_per_worker=1)
# client = Client(cluster)

In [34]:
# client

In [35]:
# # Get base distributed model
# pandda_event_model_distributed = PanDDAEventModelDistributed(pandda_config.statistical_model,
#                                       pandda_config.clusterer,
#                                       pandda_config.event_finder,
#                                         dataset=dataset,
#                                       bdc_calculator=pandda_config.bdc_calculator,
#                                       statistics=[],
#                                       map_maker=pandda_config.map_maker, 
#                                       event_table_maker=pandda_config.event_table_maker,
#                                       cpus=config["args"]["cpus"],
#                                       tree=tree)

In [36]:
# pandda_event_model_distributed.instantiate(reference,
#                                            tree)

In [37]:
# # Instantiate models
# models = [pandda_event_model_distributed.clone(dataset=d, 
#                                    name=idx)
#          for idx, d
#          in ds]

In [38]:
# models

In [39]:
# # Load model moponents
# models_loaded = client.map(load,
#                     models,
#                           pure=False)

In [40]:
# models_loaded

In [41]:
# # Load models over nodes
# models_fit = client.map(fit, 
#                            models_loaded,
#                        pure=False)


In [42]:
# models_fit

In [43]:
# # Ask models to process
# models_evaluated = client.map(evaluate, 
#                               models_fit,
#                              pure=False)


In [44]:
# models_evaluated

In [45]:
# # Ask models to criticise
# event_tables = client.map(criticise, 
#                                models_evaluated, 
#                                pure=False)


In [46]:
# event_tables

In [47]:
# event_tables_results =[e.result() for e in event_tables]

In [48]:
# client.close()

# Fi, evaluate, criticise - single dataset

In [49]:
import dask
dask.config.config
dask.config.set({"distributed.admin.tick.limit": "120s"})

In [50]:
# cluster = LocalCluster(n_workers=2, threads_per_worker=4)

In [51]:
# client = Client(cluster)

In [76]:
cluster = SGECluster(queue="medium.q",
                     cores=20,
                     processes=5,
                           memory="64GB",
                           resourcce_spec="m_mem_free=64G",
                    python="/dls/science/groups/i04-1/conor_dev/ccp4/build/bin/cctbx.python",
                    walltime="01:00:00")

/dls/science/groups/i04-1/conor_dev/ccp4/base/lib/python2.7/site-packages/distributed/bokeh/core.py:74: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [77]:
cluster.scale(5)

In [78]:
client = Client(cluster)

In [81]:
client

Client Scheduler: tcp://172.23.159.5:42513 Dashboard: http://172.23.159.5:41761/status,Cluster Workers: 5 Cores: 20 Memory: 64.00 GB


In [82]:
# Main Loop
dataloader = DefaultPanDDADataloader(min_train_datasets=60, 
                                     max_test_datasets=60)


In [83]:
ds = [(idx, d) for idx, d in dataloader(dataset)]

Got all train datasets
Sorted datasets
Collected trian datasets
Got all test datasets
sorted test datasets
yielding dataset
Dataset 0 of length 60; res limits (2.13786674777,2.13786674777)
yielding dataset
Dataset 1 of length 111; res limits (2.13849713107,2.23009496576)
yielding dataset
Dataset 2 of length 120; res limits (2.23981287927,2.31509360885)
yielding dataset
Dataset 3 of length 120; res limits (2.31816393209,2.3755079232)
yielding dataset
Dataset 4 of length 120; res limits (2.37600830711,2.45904047421)
yielding dataset
Dataset 5 of length 120; res limits (2.45982275972,2.5285230784)
yielding dataset
Dataset 6 of length 120; res limits (2.5290507321,2.58989421755)
yielding dataset
Dataset 7 of length 120; res limits (2.59023952047,2.64984944033)
yielding dataset
Dataset 8 of length 120; res limits (2.64990070685,2.71924167548)
yielding dataset
Dataset 9 of length 120; res limits (2.71972848577,2.78869368508)
yielding dataset
Dataset 10 of length 117; res limits (2.7888705626

In [84]:
# Get base distributed model
pandda_event_model_distributed = PanDDAEventModelDistributed(pandda_config.statistical_model,
                                      pandda_config.clusterer,
                                      pandda_config.event_finder,
                                        dataset=dataset,
                                      bdc_calculator=pandda_config.bdc_calculator,
                                      statistics=[],
                                      map_maker=pandda_config.map_maker, 
                                      event_table_maker=pandda_config.event_table_maker,
                                      cpus=config["args"]["cpus"],
                                      tree=tree)

In [85]:
# pandda_event_model_distributed.instantiate(reference,
#                                            tree)

In [86]:
# Instantiate models
models = [pandda_event_model_distributed.clone(dataset=d, 
                                   name=idx)
         for idx, d
         in ds]

[Errno 17] File exists: '/dls/science/groups/i04-1/conor_dev/pandda/refactored_test_dir/0'
[Errno 17] File exists: '/dls/science/groups/i04-1/conor_dev/pandda/refactored_test_dir/1'
[Errno 17] File exists: '/dls/science/groups/i04-1/conor_dev/pandda/refactored_test_dir/2'
[Errno 17] File exists: '/dls/science/groups/i04-1/conor_dev/pandda/refactored_test_dir/3'
[Errno 17] File exists: '/dls/science/groups/i04-1/conor_dev/pandda/refactored_test_dir/4'
[Errno 17] File exists: '/dls/science/groups/i04-1/conor_dev/pandda/refactored_test_dir/5'
[Errno 17] File exists: '/dls/science/groups/i04-1/conor_dev/pandda/refactored_test_dir/6'
[Errno 17] File exists: '/dls/science/groups/i04-1/conor_dev/pandda/refactored_test_dir/7'
[Errno 17] File exists: '/dls/science/groups/i04-1/conor_dev/pandda/refactored_test_dir/8'
[Errno 17] File exists: '/dls/science/groups/i04-1/conor_dev/pandda/refactored_test_dir/9'
[Errno 17] File exists: '/dls/science/groups/i04-1/conor_dev/pandda/refactored_test_dir/10

In [87]:
# Load model moponents
# model_loaded = load(models[0])

In [88]:
dsk = {}

# loop over model blocks
for model in models:
    # Get model name
    name = str(model.name)
    print(name)
    
    # Get dataset
    dtags = set(model.dataset.partition_datasets("test").keys()
                    + model.dataset.partition_datasets("train").keys()
                    )
    
    # Get resolution
    resolutions_test = max([d.data.summary.high_res for dtag, d
                                in model.dataset.partition_datasets("test").items()])
    resolutions_train = max([d.data.summary.high_res for dtag, d
                                 in model.dataset.partition_datasets("train").items()])
    max_res = max(resolutions_test, resolutions_train)
    
    dsk["{}_model".format(name)] = model
    dsk["reference"] = reference
    dsk["tree"] = tree
    
        # Load datasets
    for dtag in dtags:
        dsk["{}".format(dtag.replace("-", "_"))] = model.dataset.datasets[dtag]
    
    # Load model
#     dsk["{}_loaded_model".format(name)] = (lambda m, r, t: m.instantiate(r, t),
#                                                        "{}_model".format(name),
#                                                         "reference",
#                                                         "tree"
#                                                       )
    dsk["{}_loaded_model".format(name)] = model.instantiate(reference, tree)
    
    dsk["{}_max_res".format(name)] = max_res
    
        # Get sample loader
    sample_loader = model.dataset.sample_loader
    dsk["{}_sample_loader".format(name)] = (lambda m: m.dataset.sample_loader,
                                            "{}_loaded_model".format(name)
                                           )
    
    # ref map
    dsk["{}_ref_map".format(name)] = (lambda sl: sl.ref_map,
                                     "{}_sample_loader".format(name))
    
    # Load maps
    for dtag in dtags:
        dsk["{}_{}_map".format(name, dtag.replace("-", "_"))] = (lambda sl, r, _d: sl.get_sample(r, _d), 
                                               "{}_sample_loader".format(name),
                                               "{}_max_res".format(name), 
                                               "{}".format(dtag.replace("-", "_")))
    
    # Fit model
    dsk["{}_fit_model".format(name)] = (lambda m, train, test: m.fit(train, test),
                           "{}_loaded_model".format(name), 
                             ["{}_{}_map".format(name, dtag.replace("-", "_")) for dtag, d in model.dataset.partition_datasets("train").items()], 
                             ["{}_{}_map".format(name, dtag.replace("-", "_")) for dtag, d in model.dataset.partition_datasets("test").items()]
                          )
    
    # Find events
    for dtag in dtags:
        d = "{}".format(dtag.replace("-","_"))
        dsk["{}_{}_events".format(name, dtag.replace("-","_"))]  = (lambda m, s, _d, ref: m.evaluate_single(s, _d, ref),
                                                   "{}_fit_model".format(name),
                                                   "{}_{}_map".format(name, dtag.replace("-","_")),
                                                   d,
                                                   "{}_ref_map".format(name)
                                                  )
    
    # Criticise
    for dtag in dtags:
        d = "{}".format(dtag)
        dsk["{}_{}_event_table".format(name, dtag.replace("-","_"))] = (lambda m, _d, e: m.criticise_single(_d, e),
                                                       "{}_fit_model".format(name),
                                                       d,
                                                       "{}_{}_events".format(name, dtag.replace("-","_"))
                                                      )
    
    # Join
    dsk["{}_event_table".format(name)] = (lambda m, et: m.criticise_all(et),
                            "{}_fit_model".format(name),
                            ["{}_{}_events".format(name, dtag.replace("-","_")) for dtag in dtags])
    break
    

0


In [89]:
# dsk["0_loaded_model"]

In [90]:
# client.get(dsk, "0_loaded_model")

In [91]:
# dsk["0_sample_loader"]

In [92]:
# client.get(dsk, "0_sample_loader")

In [93]:
# client.get(dsk, "0_max_res")

In [94]:
# client.get(dsk, "PDK2_x0384")

In [95]:
# dsk["0_PDK2_x0384_map"]

In [96]:
# client.get(dsk, "0_PDK2_x0384_map")

In [97]:
dsk["0_fit_model"]

(<function __main__.<lambda>>,
 '0_loaded_model',
 ['0_PDK2_x0739_map',
  '0_PDK2_x0422_map',
  '0_PDK2_x0812_map',
  '0_PDK2_x0815_map',
  '0_PDK2_x0219_map',
  '0_PDK2_x0817_map',
  '0_PDK2_x0819_map',
  '0_PDK2_x0407_map',
  '0_PDK2_x0526_map',
  '0_PDK2_x0187_map',
  '0_PDK2_x0329_map',
  '0_PDK2_x0736_map',
  '0_PDK2_x0522_map',
  '0_PDK2_x0081_map',
  '0_PDK2_x0446_map',
  '0_PDK2_x0087_map',
  '0_PDK2_x0710_map',
  '0_PDK2_x0891_map',
  '0_PDK2_x0008_map',
  '0_PDK2_x0100_map',
  '0_PDK2_x0383_map',
  '0_PDK2_x0021_map',
  '0_PDK2_x0387_map',
  '0_PDK2_x0386_map',
  '0_PDK2_x0861_map',
  '0_PDK2_x0352_map',
  '0_PDK2_x0865_map',
  '0_PDK2_x0848_map',
  '0_PDK2_x0849_map',
  '0_PDK2_x0384_map',
  '0_PDK2_x0510_map',
  '0_PDK2_x0513_map',
  '0_PDK2_x0516_map',
  '0_PDK2_x0517_map',
  '0_PDK2_x0537_map',
  '0_PDK2_x0808_map',
  '0_PDK2_x0644_map',
  '0_PDK2_x0645_map',
  '0_PDK2_x0555_map',
  '0_PDK2_x0760_map',
  '0_PDK2_x0802_map',
  '0_PDK2_x0574_map',
  '0_PDK2_x0832_map',
  '0

In [98]:
client.get(dsk, "0_fit_model")

<pandda_analyse.event_model_distributed.PanDDAEventModelDistributed instance at 0x7f3664455290>

In [99]:
dsk["0_PDK2_x0384_events"]

(<function __main__.<lambda>>,
 '0_fit_model',
 '0_PDK2_x0384_map',
 'PDK2_x0384',
 '0_ref_map')

In [100]:
client.get(dsk, "0_PDK2_x0384_events")

AttributeError: 'NoneType' object has no attribute 'global_mask'

In [ ]:
dsk["0_PDK2_x0384_event_table"]

In [ ]:
client.get(dsk, "0_PDK2_x0384_event_table")

In [ ]:
dsk["0_event_table"]

In [ ]:
dsk[""]

In [ ]:
dsk["0_max_res"]

In [ ]:
dsk["PDK2-x0384"]

In [ ]:
client.get(dsk, "0_PDK2-x0384_map")

In [ ]:
client.get(dsk, "0_event_table")

In [ ]:
client.get(dsk, )

In [ ]:
# dsk["model_loaded"] = model_loaded

In [ ]:
dtags = set(model_loaded.dataset.partition_datasets("test").keys()
                    + model_loaded.dataset.partition_datasets("train").keys()
                    )

In [ ]:
resolutions_test = max([d.data.summary.high_res for dtag, d
                                in model_loaded.dataset.partition_datasets("test").items()])
resolutions_train = max([d.data.summary.high_res for dtag, d
                                 in model_loaded.dataset.partition_datasets("train").items()])
max_res = max(resolutions_test, resolutions_train)



In [ ]:
sample_loader = model_loaded.dataset.sample_loader

In [ ]:
# sample_loaders = {dtag: lambda d: sample_loader.get_sample(res, d)
#                           for dtag
#                           in dtags}

In [ ]:
for dtag in dtags:
    dsk[dtag] = (sample_loader.get_sample, max_res, model_loaded.dataset.datasets[dtag])

In [ ]:
dsk["params"] = (model_loaded.statistical_model.fit, 
                 [dtag for dtag, d in model_loaded.dataset.partition_datasets("train").items()], 
                 [dtag for dtag, d in model_loaded.dataset.partition_datasets("test").items()])

In [ ]:
dsk

In [ ]:
params = client.get(dsk, "params")

In [ ]:
# Load models over nodes
model_fit = client.submit(fit, model_loaded)


In [ ]:
model_fit.result()

In [ ]:
# Ask models to process
model_evaluated = evaluate(model_fit)


In [ ]:
model_evaluated.result()

In [ ]:
# Ask models to criticise
event_tables = criticise(models_evaluated)


In [ ]:
client.reset()

In [ ]:
client.restart()

In [ ]:
client.close()

# Fit, evaluate, Criticise - All datasets, Dask distributed

## TESTING

In [ ]:
cluster = SGECluster(queue="medium.q",
                     cores=1,
                     processes=1,
                           memory="64GB",
                           resourcce_spec="m_mem_free=64G",
                    python="/dls/science/groups/i04-1/conor_dev/ccp4/build/bin/cctbx.python")

In [ ]:
cluster.scale(3)

In [ ]:
time.sleep(15)

In [ ]:
cluster = LocalCluster(n_workers=2, threads_per_worker=1)

In [ ]:
client = Client(cluster)

In [ ]:
client

In [ ]:
# Main Loop
dataloader = DefaultPanDDADataloader(min_train_datasets=60, 
                                     max_test_datasets=60)


In [ ]:
ds = [(idx, d) for idx, d in dataloader(dataset)]

In [ ]:
# Get base distributed model
pandda_event_model_distributed = PanDDAEventModelDistributed(pandda_config.statistical_model,
                                      pandda_config.clusterer,
                                      pandda_config.event_finder,
                                        dataset=dataset,
                                      bdc_calculator=pandda_config.bdc_calculator,
                                      statistics=[],
                                      map_maker=pandda_config.map_maker, 
                                      event_table_maker=pandda_config.event_table_maker,
                                      cpus=config["args"]["cpus"],
                                      tree=tree)

In [ ]:
pandda_event_model_distributed.instantiate(reference,
                                           tree)

In [ ]:
# Instantiate models
models = [pandda_event_model_distributed.clone(dataset=d, 
                                   name=idx)
         for idx, d
         in ds]

In [ ]:
# models

In [ ]:
# # Load model moponents
# models_loaded = client.map(load,
#                     models,
#                           pure=False)

In [ ]:
# models_loaded

In [ ]:
# # Load models over nodes
# models_fit = client.map(fit, 
#                            models_loaded,
#                        pure=False)


In [ ]:
# models_fit

In [ ]:
# # Ask models to process
# models_evaluated = client.map(evaluate, 
#                               models_fit,
#                              pure=False)


In [ ]:
# models_evaluated

In [ ]:
# # Ask models to criticise
# event_tables = client.map(criticise, 
#                                models_evaluated, 
#                                pure=False)


In [ ]:
# event_tables

In [ ]:
# event_tables_results =[e.result() for e in event_tables]

In [ ]:
dsk = {}
for i, model in enumerate(models):
    dsk["load_{}".format(i)] = (load, model)
    dsk["fit_{}".format(i)] = (fit, "load_{}".format(i))
    dsk["evaluate_{}".format(i)] = (evaluate, "fit_{}".format(i))
    dsk["criticise_{}".format(i)] = (criticise, "evaluate_{}".format(i))


In [ ]:
dsk

In [ ]:
dsk_combined, deps = dask.optimization.fuse(dsk)

In [ ]:
client.get(dsk_combined, ["criticise_{}".format(i)
                          for i, model
                          in enumerate(models)])

In [ ]:
client.close()

In [ ]:
# Visualise tasks


In [ ]:
# Ask models for events
event_tables = event_tables.results()

In [ ]:
with ProcessModelSeriel() as P:            

    for dataset in dataloader(dataset):

        P(pandda_event_model(dataset))
        # call with self as model: model.fit(); model.evaluate(); model.criticise()
            # Seriel: run immediately
            # Qsub: pick model

    # exit: 
        # Seriel: just go on
        # qsub: wait for the jobs to complete

In [ ]:
for dataset in dataloader(dataset):
    
    with pandda_event_model(dataset) as model:

        # Fit model
        print("Fitting model")
        model.fit()  # Fit the statistical model

        # Evaluate model
        model.evaluate_parallel()  # Evaluate the fitted model on maps, fidning events

        # Criticise Model
        model.criticise() # Stores statistics from model fitting and evaluation


# Criticise Run

## TESTING

In [ ]:
# Criticise Model
pandda_statistics = PanDDARunStatistics(dataset, model)  # Generates statistics from dataset and model
PanDDARunGraphs(pandda_statistics)  # Produces a set of graphs of statistics
pandda_html = PanDDARunHTML(pandda_statistics)  # Produces a HTML from Statistics
PanDDARunLog()  # Produces a log of data processing from dataset, model, graphs and HTML


In [ ]:
with processor as P:            

    for dataset in dataloader(dataset):

        P(pandda_event_model(dataset))
        # call with self as model: model.fit(); model.evaluate(); model.criticise()
            # Seriel: run immediately
            # Qsub: pick model

    # exit: 
        # Seriel: just go on
        # qsub: wait for the jobs to complete
        

In [ ]:
class PanDDARunStatistics:
    
    def __init__(dataset, model):
        
        self.statistics = [...]
        
        for statistic in self.statistic:
            try:
                statistic.calculate()
                self.trace[statistic.name] = statistic.log()

            except Exeption as e:
                self.trace[statistic.name] = "{}".format(e)
        

In [ ]:
class OutputNativeMaps:
    
    def calculate(samples):
        
        
        
    def log():
        
    